# Imports

In [1]:
import wandb
import pandas as pd

# Guide

See here for guide https://docs.wandb.ai/guides/track/public-api-guide

# Single run

In [ ]:
run_id = "kn9o0vjo"
api = wandb.Api()
run = api.run("philipwastaken/02456_project/runs/" + run_id)
history = run.scan_history()

In [ ]:
train_dict = {'mean_train_reward':[],
              'frame_count': [],
              '_timestamp': []}

validate_dict = {'sims_per_val':[],
                 'avg_reward':[],
                 'duration':[],
                 'avg_frame_count':[]}

for row in history:
    if 'validate/sims_per_val' in row:
        validate_dict['sims_per_val'].append(row['validate/sims_per_val'])
        validate_dict['avg_reward'].append(row['validate/avg_reward'])
        validate_dict['duration'].append(row['validate/duration'])
        validate_dict['avg_frame_count'].append(row['validate/avg_frame_count'])
    else:
        train_dict['mean_train_reward'].append(row['mean_train_reward'])
        train_dict['frame_count'].append(row['frame_count'])
        train_dict['_timestamp'].append(row['_timestamp'])
        
train_df = pd.DataFrame(train_dict)
validate_df = pd.DataFrame(validate_dict)

In [ ]:
train_df

In [ ]:
validate_df

# Multiple Runs

In [2]:
api = wandb.Api() 
runs = api.runs("philipwastaken/02456_project")

### TODO:
1) Get index of runs with proper hyperparams (run.config) and didn't crash (run.summary)
2) Sort by wrapper
3) Calculate mean amongst different wrappers

In [9]:
for run in runs: 
    conf = run.config
    print(conf['update'])

{'tau': 0.01, 'gamma': 0.99, 'val_freq': 1, 'batch_size': 32, 'num_episodes': 20000, 'weight_decay': 0, 'episode_limit': 10000, 'learning_rate': 0.0003, 'prefill_memory': True, 'replay_memory_capacity': 50000}
{'tau': 0.01, 'gamma': 0.99, 'val_freq': 1, 'batch_size': 32, 'num_episodes': 20000, 'weight_decay': 0, 'episode_limit': 10000, 'learning_rate': 0.0003, 'prefill_memory': True, 'replay_memory_capacity': 50000}
{'tau': 0.01, 'gamma': 0.99, 'val_freq': 1, 'batch_size': 32, 'num_episodes': 20000, 'weight_decay': 0, 'episode_limit': 10000, 'learning_rate': 0.0003, 'prefill_memory': True, 'replay_memory_capacity': 50000}
{'tau': 0.01, 'gamma': 0.99, 'val_freq': 1, 'batch_size': 32, 'num_episodes': 20000, 'weight_decay': 0, 'episode_limit': 10000, 'learning_rate': 0.0003, 'prefill_memory': True, 'replay_memory_capacity': 50000}
{'tau': 0.01, 'gamma': 0.99, 'val_freq': 1, 'batch_size': 32, 'num_episodes': 20000, 'weight_decay': 0, 'episode_limit': 10000, 'learning_rate': 0.0003, 'prefil